In [1]:
import threading

from numba import cuda, int8
import numpy as np
import itertools
from timeit import default_timer
from datetime import timedelta


#################
# On GPU
#################

@cuda.jit
def find_solutions(A, B, C):
    pos = cuda.grid(1)

    if pos <= len(A):
        permute_pawns(A[pos - 1], B[pos - 1], C)


@cuda.jit(device=True)
def permute_pawns(static_pawns, nonstatic_pawns, results):
    nonstatic_pawns_loc = cuda.local.array(shape=10, dtype=int8)
    copy_array_elements(nonstatic_pawns, nonstatic_pawns_loc)

    permutation = cuda.local.array(shape=15, dtype=int8)

    # checks if first permutation satisfies all rules
    merge_arrays(static_pawns, nonstatic_pawns_loc, permutation)
    if rules_satisfied(permutation):
        # add found solution to results
        add_perm_to_results(permutation, results)

    first = 0
    last = len(nonstatic_pawns_loc) - 1
    while 1:
        i = last

        while 1:
            i = i - 1
            if nonstatic_pawns_loc[i] < nonstatic_pawns_loc[i + 1]:
                j = last
                while not (nonstatic_pawns_loc[i] < nonstatic_pawns_loc[j]):
                    j = j - 1
                # swap the values
                nonstatic_pawns_loc[i], nonstatic_pawns_loc[j] = nonstatic_pawns_loc[j], nonstatic_pawns_loc[i]
                swap_range(nonstatic_pawns_loc, i + 1, last + 1)
                # checks if permutation satisfies all rules
                merge_arrays(static_pawns, nonstatic_pawns_loc, permutation)
                if rules_satisfied(permutation):
                    # add found solution to results
                    add_perm_to_results(permutation, results)
                break
            if i == first:
                return


@cuda.jit(device=True)
def copy_array_elements(a, b):
    for i in range(len(b)):
        b[i] = a[i]


@cuda.jit(device=True)
def swap_range(a, start_index, end_index):
    if start_index != end_index:
        for i in range(start_index, end_index):
            diff = i - start_index
            j = end_index - 1 - diff
            if j > i:
                a[i], a[j] = a[j], a[i]
            else:
                return


@cuda.jit(device=True)
def merge_arrays(a, b, c):
    for i in range(len(a)):
        c[i] = a[i]
    for i in range(len(b)):
        c[i + len(a)] = b[i]


@cuda.jit(device=True)
def rules_satisfied(perm):
    if (perm[0] + perm[1] + perm[2] + perm[3]
            == perm[4] + perm[5] + perm[6] + perm[7]
            == perm[8] + perm[9] + perm[10] + perm[11]
            == perm[12] + perm[13] + perm[14]
            == perm[0] + perm[4] + perm[8]
            == perm[1] + perm[5] + perm[9] + perm[12]
            == perm[2] + perm[6] + perm[10] + perm[13]
            == perm[3] + perm[7] + perm[11] + perm[14]
            == perm[0] + perm[5] + perm[10] + perm[14]
            == perm[3] + perm[6] + perm[9] == 30):
        return True
    else:
        return False


@cuda.jit(device=True)
def add_perm_to_results(perm, results):
    for i in range(0, len(results)):
        if cuda.atomic.compare_and_swap(results[i], np.int32(0), np.int32(perm[0])) == 0:
            copy_array_elements(perm, results[i])
            return


#################
# On CPU
#################

def split_list(a_list, wanted_parts=1):
    length = len(a_list)
    return [a_list[i*length // wanted_parts: (i+1)*length // wanted_parts] for i in range(wanted_parts)]


def get_pawns(static_length, parts):
    pawn_list = []
    static_pawn_parts = split_list([list(t) for t in itertools.permutations(list(range(1, 16)), static_length)], parts)
    for static_pawn_part in static_pawn_parts:
        pawn_list_part = []
        nonstatic_pawn_part = []
        for s in static_pawn_part:
            nonstatic_pawns = []
            for n in range(1, 16):
                if n not in s:
                    nonstatic_pawns.append(n)
            nonstatic_pawn_part.append(nonstatic_pawns)
        pawn_list_part.append(static_pawn_part)
        pawn_list_part.append(nonstatic_pawn_part)
        pawn_list.append(pawn_list_part)
    return pawn_list


def get_pawn_string(pawns):
    s = '%02.f | %02.f | %02.f | %02.f\n' \
           '%02.f | %02.f | %02.f | %02.f\n' \
           '%02.f | %02.f | %02.f | %02.f\n' \
           '   | %02.f | %02.f | %02.f'
    return s % tuple(pawns)


# send data arrays to kernel function on the gpu
def calculate_solution(static_pawns, nonstatic_pawns, results, device_id, blocks_per_grid, threads_per_block):
    cuda.select_device(device_id)
    find_solutions[blocks_per_grid, threads_per_block](static_pawns, nonstatic_pawns, results)
    cuda.close()


#################
# Main
#################

start_total = default_timer()

# num static pawns
static_length = 5

# num gpus
num_gpus = len(cuda.gpus)

# construct inputs
pawns = get_pawns(static_length, num_gpus)
results = np.zeros(shape=(num_gpus, 1000, 15), dtype=np.int32)

threads = []

for i in range(num_gpus):
    static_pawns = np.array(pawns[i][0], dtype=np.int8)
    nonstatic_pawns = np.array(pawns[i][1], dtype=np.int8)
    total_num_threads = len(static_pawns)
    threads_per_block = 128
    blocks_per_grid = total_num_threads // threads_per_block + bool(total_num_threads % threads_per_block)
    thread = threading.Thread(target=calculate_solution, args=(static_pawns, nonstatic_pawns, results[i],
                                                               i, blocks_per_grid, threads_per_block))
    threads.append(thread)

start_gpu = default_timer()

# start all threads
for x in threads:
    x.start()

# wait for all of them to finish
for x in threads:
    x.join()

end_gpu = default_timer()

# filter solutions from zero arrays
solution_count = 0
for gpu in results:
    for r in gpu:
        if r[0] != 0:
            solution_count += 1
            print('Lösung ' + str(solution_count) + ':')
            print(get_pawn_string(r.tolist()))
            print()

end_total = default_timer()

print('\nSolutions found: ' + str(solution_count)
      + '\n\nProcessing time (GPU): ' + str(timedelta(seconds=end_gpu - start_gpu))
      + '\nProcessing time (total): ' + str(timedelta(seconds=end_total - start_total)))

Lösung 1:
03 | 08 | 04 | 15
14 | 05 | 09 | 02
13 | 06 | 10 | 01
   | 11 | 07 | 12

Lösung 2:
03 | 09 | 06 | 12
14 | 04 | 11 | 01
13 | 07 | 08 | 02
   | 10 | 05 | 15

Lösung 3:
03 | 09 | 06 | 12
13 | 04 | 11 | 02
14 | 07 | 08 | 01
   | 10 | 05 | 15

Lösung 4:
03 | 05 | 10 | 12
14 | 08 | 07 | 01
13 | 11 | 04 | 02
   | 06 | 09 | 15

Lösung 5:
03 | 04 | 10 | 13
15 | 06 | 08 | 01
12 | 09 | 07 | 02
   | 11 | 05 | 14

Lösung 6:
03 | 04 | 11 | 12
14 | 07 | 08 | 01
13 | 10 | 05 | 02
   | 09 | 06 | 15

Lösung 7:
03 | 10 | 04 | 13
12 | 05 | 11 | 02
15 | 06 | 08 | 01
   | 09 | 07 | 14

Lösung 8:
03 | 04 | 09 | 14
15 | 08 | 05 | 02
12 | 11 | 06 | 01
   | 07 | 10 | 13

Lösung 9:
03 | 05 | 08 | 14
12 | 10 | 07 | 01
15 | 09 | 04 | 02
   | 06 | 11 | 13

Lösung 10:
03 | 04 | 08 | 15
14 | 09 | 05 | 02
13 | 10 | 06 | 01
   | 07 | 11 | 12

Lösung 11:
03 | 10 | 05 | 12
14 | 04 | 11 | 01
13 | 07 | 08 | 02
   | 09 | 06 | 15

Lösung 12:
03 | 09 | 04 | 14
12 | 06 | 11 | 01
15 | 05 | 08 | 02
   | 10 | 07 | 13

L